In [1]:
import ml_mr.simulation as mr_sim
import numpy as np

In [2]:
sim = mr_sim.Simulation(100_000)

sim.add_sim_parameter("h2", 0.2)
sim.add_sim_parameter("n_variants", 15)

sim.add_sim_parameter("ux_effect", 0.15)
sim.add_sim_parameter("uy_effect", 0.1)
sim.add_sim_parameter("xy_effect", 0.07)  # Target parameter

# Noise
sim.add_sim_parameter("x_e_std", 1)
sim.add_sim_parameter("y_e_std", 1)

In [3]:
# Standard normal for the confounder
sim.add_variable(mr_sim.Normal("u", 0, 1))

# We generate the variant frequencies "outside" of
# the simulation model because it's not very important.
# This could also be done using real simulation variables
# to ensure resampling.
# We still record them as (fixed) simulation parameters.
@mr_sim.variable
def variant_frequencies(sim):
    return np.random.uniform(
    0.05,
    0.5 - 0.05,
    size=sim.get_sim_parameter("n_variants")
)
sim.add_sim_parameter(variant_frequencies)

freqs = sim.get_sim_parameter("variant_frequencies")
# We now simulate independant genetic variants.
for i in range(sim.get_sim_parameter("n_variants")):
    variant = mr_sim.Variant(f"v{i+1}", freqs[i])
    sim.add_variable(variant)

# We simulate variant effects.
sim.add_sim_parameter(
    mr_sim.Normal(
        "gx_effects",
        mu=0,
        sigma=np.sqrt(sim.get_sim_parameter("h2") / sim.get_sim_parameter("n_variants")),
        size=sim.get_sim_parameter("n_variants")
    )
)
    
# We now have everything to simulate the exposure.
@mr_sim.variable
def exposure(sim):
    variant_effects = sim.get_sim_parameter("gx_effects")
    
    x = 0
    for i in range(sim.get_sim_parameter("n_variants")):
        variant = sim.get_variable_data(f"v{i+1}")
        effect = variant_effects[i]
        
        x += effect * variant
    
    # Add effect of counfounder.
    x += sim.get_sim_parameter("ux_effect") * sim.get_variable_data("u")
    
    # Add residual noise.
    x += np.random.normal(0, scale=sim.get_sim_parameter("x_e_std"), size=sim.n)
    
    return x
    

# And the outcome.
@mr_sim.variable
def outcome(sim):
    return (
        sim.get_sim_parameter("xy_effect") * sim.get_variable_data("exposure") +
        sim.get_sim_parameter("uy_effect") * sim.get_variable_data("u") +
        np.random.normal(0, scale=sim.get_sim_parameter("y_e_std"), size=sim.n)
    )


sim.add_variable(exposure)
sim.add_variable(outcome)

In [4]:
sim.save(0)
sim.resample()
sim.save(1)